## Analyzing borrowers’ risk of defaulting

We need to prepare a report for a bank’s loan division. We’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Our report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.


### Step 1. Opening the data file and have a look at the general information. 

In [1]:
import pandas as pd
credit_scoring = pd.read_csv('https://code.s3.yandex.net/datasets/credit_scoring_eng.csv')
credit_scoring



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [2]:
credit_scoring.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
credit_scoring.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [4]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [5]:
import numpy as np
credit_scoring.describe(include=np.object)

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,secondary education,married,F,employee,wedding ceremony
freq,13750,12380,14236,11119,797


In [6]:
credit_scoring.columns
#just checking if there is any problems like spaces with column names 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Conclusion

I opened the data and from first glance we can see negative values in column days_employed. Values in columns days_employed and total_income can look better if we change data type from float to int and there are  19351 non null objects while we have 21525 entries. So we will have to see what are these values. In dob_years column we can see 0 age. Thats weird. Also in describe() we can see in children column there are values 20 children and -1, that's our next part - BIG cleaning. And we see alot of similar names of categories names in education and purpose columns. My personal fun facts from this data are mean for children 0,5 ( half of the child - that's creepy) and that the top purpose for the loan is wedding ceremony (Really?!). But we will see soon the real picture. 


### Step 2. Data preprocessing

### Processing missing values

In [7]:
#first, lets tidy up after kids
credit_scoring["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [8]:
#we will use loc.method() with the condition 20 to find the rows we need
children_20 = credit_scoring[credit_scoring['children'] ==20]
children_20

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,secondary education,1,married,0,F,employee,1,21363.842,to own a car
21325,20,-601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21404,20,-494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate


In [9]:
#we will use loc.method() with the condition -1 to find the rows we need
children_minus1 = credit_scoring[credit_scoring['children'] ==-1]
children_minus1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,secondary education,1,civil partnership,1,F,employee,0,16450.615,profile education
705,-1,-902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
742,-1,-3174.456205,57,secondary education,1,married,0,F,employee,0,10282.887,supplementary education
800,-1,349987.852217,54,secondary education,1,unmarried,4,F,retiree,0,13806.996,supplementary education
941,-1,NaN,57,Secondary Education,1,married,0,F,retiree,0,NaN,buying my own car
1363,-1,-1195.264956,55,SECONDARY EDUCATION,1,married,0,F,business,0,11128.112,profile education
1929,-1,-1461.303336,38,secondary education,1,unmarried,4,M,employee,0,17459.451,purchase of the house
2073,-1,-2539.761232,42,secondary education,1,divorced,3,F,business,0,26022.177,purchase of the house
3814,-1,-3045.290443,26,Secondary Education,1,civil partnership,1,F,civil servant,0,21102.846,having a wedding
4201,-1,-901.101738,41,secondary education,1,married,0,F,civil servant,0,36220.123,transactions with my real estate


In [10]:
credit_scoring['children'].median()
#we dont know what values were actually in the begining, so the logical thing is to replace them with median

0.0

In [11]:
#check...and no crazy 20 children now!
credit_scoring.loc[credit_scoring['children'] == 20, 'children'] = credit_scoring['children'].median()
children_20 = credit_scoring[credit_scoring['children'] ==20]
children_20

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [12]:
#check...and no crazy -1 children now!
credit_scoring.loc[credit_scoring['children'] == -1, 'children'] = credit_scoring['children'].median()
children_minus1 = credit_scoring[credit_scoring['children'] ==-1]
children_minus1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [13]:
#now we got to our missing values
credit_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [14]:
credit_scoring[credit_scoring.total_income.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0.0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0.0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0.0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0.0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0.0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2.0,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1.0,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0.0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1.0,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [15]:
credit_scoring['total_income'].value_counts(dropna = False)

NaN          2174
17312.717       2
31791.384       2
42413.096       2
26935.722       1
             ... 
48796.341       1
34774.610       1
15710.698       1
19232.334       1
29653.643       1
Name: total_income, Length: 19349, dtype: int64

In [16]:
#we will replace the NaNs in total_income column with median
credit_scoring['total_income'].median()

23202.87

In [17]:
credit_scoring['total_income'].fillna(value = ('23202.87'), inplace = True)

In [18]:
#check...and no Nans anymore!
credit_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [19]:
#we will do the same replacing with Nans in days_employed
credit_scoring[credit_scoring.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0.0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,23202.87,to have a wedding
26,0.0,NaN,41,secondary education,1,married,0,M,civil servant,0,23202.87,education
29,0.0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,23202.87,building a real estate
41,0.0,NaN,50,secondary education,1,married,0,F,civil servant,0,23202.87,second-hand car purchase
55,0.0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,23202.87,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2.0,NaN,47,Secondary Education,1,married,0,M,business,0,23202.87,purchase of a car
21495,1.0,NaN,50,secondary education,1,civil partnership,1,F,employee,0,23202.87,wedding ceremony
21497,0.0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,23202.87,building a property
21502,1.0,NaN,42,secondary education,1,married,0,F,employee,0,23202.87,building a real estate


In [20]:
#but 1st we will get rid of negative energies in days_employed column
credit_scoring['days_employed'] = credit_scoring['days_employed'].abs()

In [21]:
#check...and thigs became much more positive here!
credit_scoring['days_employed'].head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

In [22]:
#we will replace all NaNs with median also in days_employed column
credit_scoring['days_employed'].median()

2194.220566878695

In [23]:
credit_scoring['days_employed'].fillna(value = ('2194.220566878695'), inplace = True)

In [24]:
#check...and no NaNs in days_employed column anymore!
credit_scoring[credit_scoring.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [25]:
#just checking also for 0s
credit_scoring.loc[credit_scoring['days_employed'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [26]:
#all clean, Captain!
credit_scoring.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [27]:
#nope!we dont want to give a loan to newborns
dob_years_0= credit_scoring[credit_scoring['dob_years'] ==0]
dob_years_0

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0.0,346542,0,Secondary Education,1,married,0,F,retiree,0,11406.6,car
149,0.0,2664.27,0,secondary education,1,divorced,3,F,employee,0,11228.2,housing transactions
270,3.0,1872.66,0,secondary education,1,married,0,F,employee,0,16346.6,housing renovation
578,0.0,397857,0,secondary education,1,married,0,F,retiree,0,15619.3,construction of own property
1040,0.0,1158.03,0,bachelor's degree,0,divorced,3,F,business,0,48639.1,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0.0,2194.220566878695,0,secondary education,1,married,0,F,employee,0,23202.87,housing
20462,0.0,338735,0,secondary education,1,married,0,F,retiree,0,41471,purchase of my own house
20577,0.0,331741,0,secondary education,1,unmarried,4,F,retiree,0,20766.2,property
21179,2.0,108.967,0,bachelor's degree,0,married,0,M,business,0,38512.3,building a real estate


In [28]:
#we will change 0s also with median
credit_scoring['dob_years'].median()

42.0

In [29]:
credit_scoring.loc[credit_scoring['dob_years'] == 0, 'dob_years'] = credit_scoring['dob_years'].median()

In [30]:
#check...and no 0s anymore in dob_years column
dob_years_0= credit_scoring[credit_scoring['dob_years'] ==0]
dob_years_0

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


### Conclusion

The best way to deal with missing values or weird values is to contact the data provider, but not always we have this opportunity. In our case we mostly took a decision of replacing detected missing values NaNs and misplaced 0s and other weird numbers with median not to loose the big amount of usefull data.

### Data type replacement

In [31]:
credit_scoring.dtypes

children            float64
days_employed        object
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income         object
purpose              object
dtype: object

In [32]:
#we use astype method 
credit_scoring['children'] = credit_scoring['children'].astype('int')
credit_scoring['children'].dtypes

dtype('int64')

In [33]:
#we use astype method 
credit_scoring['dob_years'] = credit_scoring['dob_years'].astype('int')
credit_scoring['dob_years'].dtypes

dtype('int64')

In [34]:
#we use tonumeric method to change object type to numeric and astype method 
credit_scoring['total_income']= pd.to_numeric(credit_scoring['total_income'], errors= 'ignore')
credit_scoring['total_income'].value_counts(dropna = False)


23202.870    2175
17312.717       2
42413.096       2
31791.384       2
15830.634       1
             ... 
48796.341       1
34774.610       1
15710.698       1
19232.334       1
9591.824        1
Name: total_income, Length: 19348, dtype: int64

In [35]:
credit_scoring['total_income'] = credit_scoring['total_income'].astype('int')
credit_scoring['total_income'].dtypes

dtype('int64')

In [36]:
credit_scoring['days_employed']= pd.to_numeric(credit_scoring['days_employed'], errors='coerce')
credit_scoring['days_employed'].value_counts(dropna = False)


2194.220567    2175
986.927316        1
1893.222792       1
4236.274243       1
6620.396473       1
               ... 
2849.351119       1
5619.328204       1
448.829898        1
1687.038672       1
582.538413        1
Name: days_employed, Length: 19351, dtype: int64

In [37]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].astype('int')
credit_scoring['days_employed'].dtypes

dtype('int64')

In [38]:
credit_scoring['days_employed']

0          8437
1          4024
2          5623
3          4124
4        340266
          ...  
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, Length: 21525, dtype: int64

In [39]:
# when we changed days_employed to int type we can see that values are very different from each other
print(credit_scoring.days_employed.min())
print(credit_scoring.days_employed.max())
# i wonder what are these...days? Please, no! Then someone was working max 1100 years and min 24 days, if its hours then max is 45,8 years and min is 24 hours
# as we dont need days_employed column for our final answers, maybe we leave it as it is

24
401755


In [40]:
#check...and we got all int types we need

In [41]:
credit_scoring.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Conclusion

We changed object type to int in case we have seen it supposed to be numbers for possible arithmetic operations. And float type to int for numbers to be more accurate and data look cleaner.

### Processing duplicates

In [42]:
#we are fixing the mess in value names with str. lower() method
credit_scoring['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [43]:
credit_scoring['education'] = credit_scoring['education'].str.lower()
credit_scoring['education'].value_counts()


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [44]:
#now we can see what's up with duplicates
credit_scoring.loc[credit_scoring.duplicated(), :]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,secondary education,1,married,0,F,employee,0,23202,purchase of the house for my family
3290,0,2194,58,secondary education,1,civil partnership,1,F,retiree,0,23202,to have a wedding
4182,1,2194,34,bachelor's degree,0,civil partnership,1,F,employee,0,23202,wedding ceremony
4851,0,2194,60,secondary education,1,civil partnership,1,F,retiree,0,23202,wedding ceremony
5557,0,2194,58,secondary education,1,civil partnership,1,F,retiree,0,23202,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,2194,64,secondary education,1,married,0,F,retiree,0,23202,supplementary education
21032,0,2194,60,secondary education,1,married,0,F,retiree,0,23202,to become educated
21132,0,2194,47,secondary education,1,married,0,F,employee,0,23202,housing renovation
21281,1,2194,30,bachelor's degree,0,married,0,F,employee,0,23202,buy commercial real estate


In [45]:
credit_scoring.shape

(21525, 12)

In [46]:
#we are dropping duplicates with drop method
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop = True)
credit_scoring.shape

(21453, 12)

In [47]:
credit_scoring.duplicated().sum()

0

### Conclusion

Maybe it could appear more duplicates after we replaced missing values with meadians. But what i noticed that if i would dropped duplicates before dealing with column education there would still be some duplicates. So the right decision is to drop them after everything is done.

### Categorizing Data

In [48]:
#now we ll deal with categories of purpose column
credit_scoring['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            767
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [49]:
# at first we need to get stemms of all expressions we have in the column
from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')
list_words = ['wedding', 'estate', 'housing', 'car', 'property', 'university', 'education']
for word in list_words:
    print('Source word - {}, after stemming - {}'.format(word, english_stemmer.stem(word)))
    

Source word - wedding, after stemming - wed
Source word - estate, after stemming - estat
Source word - housing, after stemming - hous
Source word - car, after stemming - car
Source word - property, after stemming - properti
Source word - university, after stemming - univers
Source word - education, after stemming - educ


In [50]:
# we design the function that returns new category name in case of meeting the condition of a stemmed word
from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')

def loan_purpose(data):
    for purpose in data:
        for word in data.split(" "):
            stemmed_word = english_stemmer.stem(word)
            if stemmed_word == 'estat' or stemmed_word == 'hous' or stemmed_word == 'properti':
                return 'real estate'
            elif stemmed_word == 'wed':
                return 'wedding'
            elif stemmed_word == 'car':
                return 'vehicle'
            elif stemmed_word == 'educ' or stemmed_word== 'univers':
                return 'education'
            
           


    
        

print(loan_purpose('to buy a car'))
print(loan_purpose('profile education'))
print(loan_purpose('going to university'))
print(loan_purpose('building a property'))
print(loan_purpose('housing'))
print(loan_purpose('wedding ceremony'))
print(loan_purpose('buy commercial real estate'))

vehicle
education
education
real estate
real estate
wedding
real estate


In [51]:
# we apply the function to the column purpose and storing new values in the new column clean_purpose
credit_scoring['clean_purpose'] = credit_scoring['purpose'].apply(loan_purpose)
credit_scoring

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,real estate
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,vehicle
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,real estate
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966,housing transactions,real estate
21449,0,343937,67,secondary education,1,married,0,F,retiree,0,24959,purchase of a car,vehicle
21450,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347,property,real estate
21451,3,3112,38,secondary education,1,married,0,M,employee,1,39054,buying my own car,vehicle


In [52]:
#check 
credit_scoring['clean_purpose'].value_counts().sum()

21453

In [53]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21453.000000,21453.000000,21453.000000,21453.000000,21453.000000,21453.000000,21453.000000
mean,0.471309,60572.835734,43.469025,0.817089,0.973896,0.081154,26435.800121
std,0.751082,133438.275373,12.214162,0.548686,1.421601,0.273078,15683.720829
min,0.000000,24.000000,19.000000,0.000000,0.000000,0.000000,3306.000000
25%,0.000000,1023.000000,33.000000,1.000000,0.000000,0.000000,17219.000000
50%,0.000000,2194.000000,42.000000,1.000000,0.000000,0.000000,23202.000000
75%,1.000000,4797.000000,53.000000,1.000000,1.000000,0.000000,31331.000000
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,362496.000000


In [54]:
## we want to categorise income values in low, medium and high income categories by using the method .quantile
def income_level(total_income):
    if total_income <= credit_scoring['total_income'].quantile(.25):
        return 'very low income'
    if  credit_scoring['total_income'].quantile(.25) < total_income <= credit_scoring['total_income'].quantile(.50):
        return 'low income'
    if credit_scoring['total_income'].quantile(.50) < total_income <= credit_scoring['total_income'].quantile(.75):
        return 'medium income'
    if total_income > credit_scoring['total_income'].quantile(.75):
        return 'high income'

    

In [55]:
print(income_level(5000))
print(income_level(20000))
print(income_level(30000))
print(income_level(50000))

very low income
low income
medium income
high income


In [56]:
#we apply the function to the column total_income and storing new values in the new column income_level
credit_scoring['income_level'] = credit_scoring['total_income'].apply(income_level)
credit_scoring

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_purpose,income_level
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,real estate,high income
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,vehicle,low income
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,real estate,medium income
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,education,high income
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,wedding,medium income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966,housing transactions,real estate,high income
21449,0,343937,67,secondary education,1,married,0,F,retiree,0,24959,purchase of a car,vehicle,medium income
21450,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347,property,real estate,very low income
21451,3,3112,38,secondary education,1,married,0,M,employee,1,39054,buying my own car,vehicle,high income


In [57]:
# we want to categorise children values in groups: no children, medium(1-2 children) and large(3 and up) family categories
def group_children(children):
    
    
    if children > 0 and children <= 2:
        return 'medium family(1&2)'
    if children == 0:
        return 'no children'
    if children >= 3:
        return 'large family(3>=)'


    
    
print(group_children(3))

large family(3>=)


In [58]:
#we apply the function to the column children and storing new values in the new column group_children
credit_scoring['group_children'] = credit_scoring['children'].apply(group_children)
credit_scoring

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_purpose,income_level,group_children
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,real estate,high income,medium family(1&2)
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,vehicle,low income,medium family(1&2)
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,real estate,medium income,no children
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,education,high income,large family(3>=)
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,wedding,medium income,no children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966,housing transactions,real estate,high income,medium family(1&2)
21449,0,343937,67,secondary education,1,married,0,F,retiree,0,24959,purchase of a car,vehicle,medium income,no children
21450,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347,property,real estate,very low income,medium family(1&2)
21451,3,3112,38,secondary education,1,married,0,M,employee,1,39054,buying my own car,vehicle,high income,large family(3>=)


### Conclusion

We categorized the numerical columns we need so now we can answer the final questions. 
And also i played around with grouping children in different ways, but left categories: 0- No children, 1-2 - medium family, 3-5 - large family

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [59]:
#we ll look our debts in children categories 
credit_scoring.groupby('debt').children.value_counts()

debt  children
0     0           13141
      1            4364
      2            1858
      3             303
      4              37
      5               9
1     0            1072
      1             444
      2             194
      3              27
      4               4
Name: children, dtype: int64

In [60]:
credit_scoring.children.value_counts()

0    14213
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [61]:
# to find a default ratio we need to devide the how many ppl defaulted debts in every children category to total ppl in each category
cs_children = credit_scoring.groupby('group_children')['debt'].agg(['count','sum'])
cs_children

,count,sum
group_children,,
large family(3>=),380,31
medium family(1&2),6860,638
no children,14213,1072


In [62]:
cs_children['default_rate_children'] = cs_children['sum'] / cs_children['count']

cs_children['default_rate_children'].sort_values()


group_children
no children           0.075424
large family(3>=)     0.081579
medium family(1&2)    0.093003
Name: default_rate_children, dtype: float64

In [63]:
#i am sure there is a way to print all the rate(mean) column of the group in once, i tried, but then did it manually
print('Default rate - 0 children: {:.1%}'.format(0.075424))
print('Default rate - large family(3>=): {:.1%}'.format(0.081579))
print('Default rate - medium family(1&2): {:.1%}'.format(0.093003))


Default rate - 0 children: 7.5%
Default rate - large family(3>=): 8.2%
Default rate - medium family(1&2): 9.3%


### Conclusion

Having now default rates, we can see that the best category in repaying on-time is 0 children families - 7.5%, after them are large families - 8.2%, and not really trying to be best in repaying is medium families - 9.3%. So, basically, people without children are more reliable then with children in our case. But people with 3 or more kids, especially, all 9 from 9 families with 5 children we had in our data who payed the debt on time, are more reliable then with 1 or 2 children. We don't know all the details, but maybe the more children you have, the better crediting programm you get, or maybe there is some part coverage from the government, or from the law perspectives it's not so easy to kick you out of the only shelter you've got with 5 kids. If we would need, we could investigate more info and figure out why these rates are like this.

- Is there a relation between marital status and repaying a loan on time?

In [64]:
credit_scoring.groupby('debt').family_status.value_counts()

debt  family_status    
0     married              11408
      civil partnership     3762
      unmarried             2536
      divorced              1110
      widow / widower        896
1     married                931
      civil partnership      388
      unmarried              274
      divorced                85
      widow / widower         63
Name: family_status, dtype: int64

In [65]:
# deviding number of ppl who defaulted debt to total number of category is basically the same that counting mean 
cs_family_status = credit_scoring.groupby('family_status')['debt'].agg(['count', 'sum', 'mean'])
cs_family_status

,count,sum,mean
family_status,,,
civil partnership,4150,388,0.093494
divorced,1195,85,0.071130
married,12339,931,0.075452
unmarried,2810,274,0.097509
widow / widower,959,63,0.065693


In [66]:
cs_family_status['mean'].sort_values()

family_status
widow / widower      0.065693
divorced             0.071130
married              0.075452
civil partnership    0.093494
unmarried            0.097509
Name: mean, dtype: float64

In [67]:
print('Default rate - widow / widower: {:.1%}'.format(0.065693))
print('Default rate - divorced: {:.1%}'.format(0.071130))
print('Default rate - married: {:.1%}'.format(0.075452))
print('Default rate - civil partnership: {:.1%}'.format(0.093494))
print('Default rate - unmarried {:.1%}'.format(0.097509))


Default rate - widow / widower: 6.6%
Default rate - divorced: 7.1%
Default rate - married: 7.5%
Default rate - civil partnership: 9.3%
Default rate - unmarried 9.8%


### Conclusion

Looking at default rates we can see that the most reliable groups are widow/widower and divorced. Here we can just make a bit crazy theory that after divorce or the death of the spouse people get a big amount of money from divorcement or insurance company, so some of them can repay the loan. After them married category is doing not bad. But the civil partnership category is not as good maried. And the last and the least good in repaying are unmarried people. So we can make a conclusion that people in the relationship are more reliable then single ones.

- Is there a relation between income level and repaying a loan on time?

In [68]:
credit_scoring.groupby('debt').income_level.value_counts()

debt  income_level   
0     low income         5867
      high income        4979
      very low income    4937
      medium income      3929
1     low income          547
      very low income     427
      medium income       384
      high income         383
Name: income_level, dtype: int64

In [69]:
cs_income = credit_scoring.groupby('income_level')['debt'].agg(['count','sum', 'mean'])
cs_income

,count,sum,mean
income_level,,,
high income,5362,383,0.071429
low income,6414,547,0.085282
medium income,4313,384,0.089033
very low income,5364,427,0.079605


In [70]:
cs_income['mean'].sort_values()

income_level
high income        0.071429
very low income    0.079605
low income         0.085282
medium income      0.089033
Name: mean, dtype: float64

In [71]:
print('Default rate - high income {:.1%}'.format(0.071429))
print('Default rate - very low income {:.1%}'.format(0.079605))
print('Default rate - low income {:.1%}'.format(0.085282))
print('Default rate - medium income {:.1%}'.format(0.089033))

Default rate - high income 7.1%
Default rate - very low income 8.0%
Default rate - low income 8.5%
Default rate - medium income 8.9%


### Conclusion

Having default rates with income level we can see that the lowest rate has category high income. No doubts!
But then we see very low and low income categories. And the highest default rate is medium income category. My assumption is that people with low income can be checked deeper before the loan is approved and can get the loan amount that they are likely to repay consided on their low income level.

- How do different loan purposes affect on-time repayment of the loan?

In [72]:
credit_scoring.groupby('debt').clean_purpose.value_counts()

debt  clean_purpose
0     real estate      10029
      vehicle           3903
      education         3643
      wedding           2137
1     real estate        782
      vehicle            403
      education          370
      wedding            186
Name: clean_purpose, dtype: int64

In [73]:
cs_purpose = credit_scoring.groupby('clean_purpose')['debt'].agg(['count','sum', 'mean'])
cs_purpose

,count,sum,mean
clean_purpose,,,
education,4013,370,0.092200
real estate,10811,782,0.072334
vehicle,4306,403,0.093590
wedding,2323,186,0.080069


In [74]:
cs_purpose['mean'].sort_values()

clean_purpose
real estate    0.072334
wedding        0.080069
education      0.092200
vehicle        0.093590
Name: mean, dtype: float64

In [75]:
print('Default rate - purpose - real estate {:.1%}'.format(0.072334))
print('Default rate - purpose - wedding {:.1%}'.format(0.080069))
print('Default rate - purpose - education {:.1%}'.format(0.092200))
print('Default rate - purpose - vehicle {:.1%}'.format(0.093590))

Default rate - purpose - real estate 7.2%
Default rate - purpose - wedding 8.0%
Default rate - purpose - education 9.2%
Default rate - purpose - vehicle 9.4%


### Conclusion

Looking at default rates by purpose we can see that real estate is the most well repaid one. The place you live in is a high importance in life values. After it comes the wedding category. Here i can assume that it may be not a big sum of the loan and people can manage repay the loan before they divorce). After education and vehicle - in these categories something can go wrong easily: loans for eduaction mostly needed for young people who can change their mind or studying can be more difficult then was expected or the young person can overrate their possibility to work and study in the same time if they are; or the car accident can lead to injuries or deth and the loan taker could not have the health insurance in these cases.

### Step 4. General conclusion

After all work with data, we came to main numbers and deductions that can influence credit score of a potential loaner:

- widow/widower and divorced people have the lowest risk in nonrepayment
- the bank can trust to people in the relationship, especially offiacilly married - they are  more reliable then single ones

- people without children have the lowest risk in nonrepayment
- people with 3 or more children are more leliable repayers then with 1 or 2 children

### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.